<a href="https://colab.research.google.com/github/NatarajanLalgudi/100-days-of-deep-learning/blob/main/Spaceship_Titanic_Kaggle_Featuretools_Vanilla_Random_Forest_29_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c spaceship-titanic
!dir

100% 299k/299k [00:00<00:00, 649kB/s]
100% 299k/299k [00:00<00:00, 648kB/s]
kaggle.json  sample_data  spaceship-titanic.zip


In [ ]:
! unzip -q spaceship-titanic.zip
!dir

kaggle.json  sample_submission.csv  test.csv
sample_data  spaceship-titanic.zip  train.csv


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

train.shape, test.shape, sub.shape

((8693, 14), (4277, 13), (4277, 2))

In [ ]:
sub.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [ ]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [ ]:
train.isna().sum().sort_values(ascending = False)

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64

In [ ]:
dfs_org = [train, test]

https://stackoverflow.com/questions/18674064/how-do-i-insert-a-column-at-a-specific-column-index-in-pandas

In [ ]:
for df in dfs_org:
    df.insert(1, 'GroupID',  df['PassengerId'].str[:4])
    df.insert(2, 'NumberID', df['PassengerId'].str[5:])

In [ ]:
spend_cols = list(train.columns[9:14])
spend_cols

['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

https://stackoverflow.com/questions/34023918/make-new-column-in-panda-dataframe-by-adding-values-from-other-columns

In [ ]:
for df in dfs_org:
    for col in spend_cols:
        df[col].fillna(0, inplace = True)
    df.insert(list(train.columns).index('VRDeck')+1, 'TotalBill', df[spend_cols].sum(axis=1))

In [ ]:
for df in dfs_org:
    df['CryoSleep'] = np.where((df['CryoSleep'].isna() & df['TotalBill'] == 0), True, df['CryoSleep'])

In [ ]:
cabinpos = list(train.columns).index('Cabin')

for df in dfs_org:
    df.insert(cabinpos+1, 'Deck', df['Cabin'].str.split('/').str[0])
    df.insert(cabinpos+2, 'CabinNumber', df['Cabin'].str.split('/').str[1])
    df.insert(cabinpos+3, 'Side', df['Cabin'].str.split('/').str[2])

In [ ]:
train_df = train.iloc[:,:-1]
target = train['Transported']
test_df = test.copy()

dfs_new = [train_df, test_df]

In [ ]:
for df in dfs_new:
    df.drop(['Cabin', 'Name', 'PassengerId'], axis = 1, inplace = True)

In [ ]:
for df in dfs_new:
    for col in ['CryoSleep', 'VIP']:
        df[col].fillna(0, inplace = True)
        df[col] = df[col].astype(int)

In [ ]:
numcols = list(train_df.select_dtypes(include = ['int','float']))
catcols = list(train_df.select_dtypes(include = ['object','category']))

In [ ]:
for col in numcols:
    print(col, train_df[col].nunique())

CryoSleep 2
Age 80
VIP 2
RoomService 1273
FoodCourt 1507
ShoppingMall 1115
Spa 1327
VRDeck 1306
TotalBill 2336


In [ ]:
for col in catcols:
    print(col, train_df[col].nunique())

GroupID 6217
NumberID 8
HomePlanet 3
Deck 8
CabinNumber 1817
Side 2
Destination 3


In [ ]:
pd.crosstab(train_df['HomePlanet'],train_df['NumberID'], normalize = 'all')

NumberID,01,02,03,04,05,06,07,08
HomePlanet,,,,,,,,
Earth,0.423104,0.061823,0.027673,0.010716,0.007890,0.005535,0.003650,0.001531
Europa,0.148375,0.060881,0.022256,0.011187,0.005299,0.002120,0.000824,0.000000
Mars,0.144018,0.039331,0.015897,0.004593,0.001649,0.000824,0.000824,0.000000


In [ ]:
for df in dfs_new:
    df.drop(['GroupID','CabinNumber'], axis = 1, inplace = True)

In [ ]:
numcols = list(train_df.select_dtypes(include = ['int','float']))
catcols = list(train_df.select_dtypes(include = ['object','category']))

In [ ]:
for df in dfs_new:
    for col in catcols:
        df[col].fillna(df[col].mode().iloc[0], inplace = True)
    for col in numcols:
        df[col].fillna(df[col].median(), inplace = True)

In [ ]:
train_df.shape, test_df.shape

((8693, 14), (4277, 14))

In [ ]:
train_df = pd.get_dummies(train_df).copy()
test_df = pd.get_dummies(test_df).copy()

In [ ]:
train_df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,NumberID_01,NumberID_02,NumberID_03,NumberID_04,NumberID_05,NumberID_06,NumberID_07,NumberID_08,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1,39.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1
1,1,24.0,0,109.0,9.0,25.0,549.0,44.0,736.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
2,1,58.0,1,43.0,3576.0,0.0,6715.0,49.0,10383.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1
3,1,33.0,0,0.0,1283.0,371.0,3329.0,193.0,5176.0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1
4,1,16.0,0,303.0,70.0,151.0,565.0,2.0,1091.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1


In [ ]:
!pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.3/618.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = train_df
y = LabelEncoder().fit_transform(target)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

X_train.shape, X_val.shape, test_df.shape

((6954, 33), (1739, 33), (4277, 33))

In [ ]:
train_data = X_train.copy()
train_data.shape

(6954, 33)

https://stackoverflow.com/questions/69665765/featuretoolsentityset-object-has-no-attribute-entity-from-dataframe

In [ ]:
import featuretools as ft

es = ft.EntitySet(id = 'Titanic')
train_data['PassengerId'] = train_data.index.map(train['PassengerId'])

In [ ]:
# Step 1: Create an EntitySet
es = ft.EntitySet(id="train_data")

# Step 2: Create an entity using the 'train_data' DataFrame
es = es.add_dataframe(
    dataframe_name="train_data",
    dataframe=train_data,
    index="PassengerId"
)

# Step 3: Define the target entity name
target_entity_name = "train_data"

# Step 4: Create new features using Deep Feature Synthesis (DFS)
# We will create features using the built-in "agg_primitives" and "trans_primitives".

# Specify aggregation primitives
agg_primitives = ["count", "mean", "sum", "max", "min", "std"]

# Specify transformation primitives
trans_primitives = ["add_numeric", "subtract_numeric", "multiply_numeric", "divide_numeric"]

# Run Deep Feature Synthesis
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                     target_dataframe_name=target_entity_name,
                                     agg_primitives=agg_primitives,
                                     trans_primitives=trans_primitives,
                                     verbose=True)

# The resulting 'feature_matrix' will contain the newly created features.
# 'feature_defs' contains the definitions of the generated features, which can be helpful to interpret the results.

# Display the feature matrix
print(feature_matrix)


Built 2673 features
Elapsed: 00:10 | Progress: 100%|██████████


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from sklearn.feature_selection import SelectKBest

# Assuming 'target_column' is the name of the target column ('Transported') in the 'feature_matrix' DataFrame
X = feature_matrix
y = y_train

In [ ]:
X.shape, y.shape

((6954, 2673), (6954,))

In [ ]:
X = X.dropna(axis=1)
X.isnull().sum().sum()

0

In [ ]:
X.shape, y.shape

((6954, 1623), (6954,))

In [ ]:
from sklearn.feature_selection import SelectFromModel, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
print("Infinite values in X:", np.any(np.isinf(X)))

Infinite values in X: True


In [ ]:
# Check for infinity values in the feature matrix X
is_inf_column = np.any(np.isinf(X), axis=0)
is_inf_record = np.any(np.isinf(X), axis=1)

# Get the names of columns with infinity values
columns_with_inf = X.columns[is_inf_column]

# Get the indices of records with infinity values
records_with_inf = X.index[is_inf_record]

# Print the columns and records with infinity values
print("Columns with infinity values:")
print(columns_with_inf)

print("\nRecords with infinity values:")
print(records_with_inf)


Columns with infinity values:
Index(['Age / CryoSleep', 'CryoSleep / Age', 'CryoSleep / TotalBill',
       'Side_P / Side_S', 'Side_S / Side_P', 'TotalBill / CryoSleep'],
      dtype='object')

Records with infinity values:
Int64Index([386804, 133601, 919301, 542201, 507601, 572801,  81103, 295301,
             84902, 285701,
            ...
            133501, 306002, 479101, 514201, 385005, 436201, 467801, 757901,
             46001, 846201],
           dtype='int64', name='PassengerId', length=6954)


In [ ]:
X.drop(columns_with_inf, axis = 1, inplace = True)

X.shape, y.shape

((6954, 1617), (6954,))

In [ ]:
scaler = StandardScaler()
scaler.fit(X)

sel_ = SelectFromModel(
    LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))
sel_.fit(scaler.transform(X), y)

unimportant_cols = X.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
len(unimportant_cols)

1331

In [ ]:
X.drop(unimportant_cols, axis = 1, inplace = True)

X.shape, y.shape

((6954, 286), (6954,))

In [ ]:
X.columns

Index(['CryoSleep', 'CryoSleep + Deck_T', 'CryoSleep + HomePlanet_Europa',
       'CryoSleep + NumberID_03', 'CryoSleep + NumberID_04',
       'CryoSleep + NumberID_08', 'Deck_E + HomePlanet_Earth',
       'Deck_G + TotalBill', 'Destination_PSO J318.5-22 + HomePlanet_Europa',
       'Destination_PSO J318.5-22 + RoomService',
       ...
       'Destination_TRAPPIST-1e - HomePlanet_Europa',
       'Destination_TRAPPIST-1e - NumberID_03', 'FoodCourt - TotalBill',
       'HomePlanet_Earth - NumberID_03', 'HomePlanet_Earth - Side_S',
       'HomePlanet_Europa - Side_P', 'HomePlanet_Europa - Spa',
       'NumberID_03 - Side_P', 'NumberID_04 - Spa', 'Spa - VIP'],
      dtype='object', length=286)

In [ ]:
predictors = list(X.columns)
mutual_info = list(mutual_info_classif(X, y, discrete_features = 'auto', random_state = 42))
mi_dict = {'Feature' : predictors, 'Mutual_Info': mutual_info}
mi = pd.DataFrame(mi_dict)
mi = mi.sort_values(by = 'Mutual_Info', ascending = False)
mi

,Feature,Mutual_Info
278,FoodCourt - TotalBill,0.159601
7,Deck_G + TotalBill,0.154514
19,RoomService + Spa,0.146578
21,RoomService + VRDeck,0.127253
20,RoomService + TotalBill,0.121013
24,Spa + VRDeck,0.120552
22,Spa + TotalBill,0.113255
282,HomePlanet_Europa - Spa,0.103778
270,Deck_B - Spa,0.095650
40,Age * Spa,0.077306


In [ ]:
low_mi_cols = mi[mi['Mutual_Info'] < 0.01]['Feature'].values.tolist()
len(low_mi_cols)

185

In [ ]:
X.drop(low_mi_cols, axis = 1, inplace = True)

In [ ]:
final_selected_features = list(X.columns)
len(final_selected_features)

101

In [ ]:
for df in [X_train, X_val, test_df]:
    # Create new features based on interactions and transformations
    for feature in final_selected_features:
        if ' + ' in feature:
            col1, col2 = feature.split(' + ')
            df[feature] = df[col1] + df[col2]
        elif ' - ' in feature:
            col1, col2 = feature.split(' - ')
            df[feature] = df[col1] - df[col2]
        elif ' * ' in feature:
            col1, col2 = feature.split(' * ')
            df[feature] = df[col1] * df[col2]
        else:
            # Handle cases where the feature doesn't contain any of the expected separators
            # You can choose to skip or apply a different operation based on your specific needs
            pass


In [ ]:
X_train.shape, X_val.shape, test_df.shape

((6954, 134), (1739, 134), (4277, 134))

In [ ]:
predictors = list(X_train.columns)
mutual_info = list(mutual_info_classif(X_train, y_train, discrete_features = 'auto', random_state = 42))
mi_dict = {'Feature' : predictors, 'Mutual_Info': mutual_info}
mi = pd.DataFrame(mi_dict)
mi = mi.sort_values(by = 'Mutual_Info', ascending = False)
mi

,Feature,Mutual_Info
37,Deck_G + TotalBill,0.155122
128,FoodCourt - TotalBill,0.153756
49,RoomService + Spa,0.138922
8,TotalBill,0.132276
51,RoomService + VRDeck,0.127277
54,Spa + VRDeck,0.118033
50,RoomService + TotalBill,0.114010
52,Spa + TotalBill,0.111507
131,HomePlanet_Europa - Spa,0.108023
120,Deck_B - Spa,0.092646


In [ ]:
low_mi_cols = mi[mi['Mutual_Info'] < 0.05]['Feature'].values.tolist()
len(low_mi_cols)

105

In [ ]:
for df in [X_train, X_val, test_df]:
    df.drop(low_mi_cols, axis = 1, inplace = True)

In [ ]:
X_train.shape, X_val.shape, test_df.shape

((6954, 29), (1739, 29), (4277, 29))

In [ ]:
from sklearn.metrics import *

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 100, random_state = 13, max_depth = 6,
                             criterion = 'gini', max_features = 'sqrt')

rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_val)

rfc_accuracy = accuracy_score(y_val, rfc_pred)

rfc_accuracy

0.8010350776308223

In [ ]:
rfc

RandomForestClassifier(max_depth=6, random_state=13)

In [ ]:
rfcpred = rfc.predict(test_df)
sub['Transported'] = rfcpred.astype("bool")
sub.to_csv("RF_Vanilla.csv",index=False)

Test Accuracy Score for RandomForestClassifier(max_depth=6, random_state=13) was 0.79775